### 配合課程影片 hmwu_StatR-09.2
### Exercise 9-2

In [1]:
!pip install imblearn

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.16.3 requires msgpack-python, which is not installed.
quandl 3.2.0 has requirement requests<2.18,>=2.7.0, but you'll have requests 2.19.1 which is incompatible.
jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.5 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from sklearn import datasets
from scipy import stats
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [3]:
ubIonosphere = pd.read_csv('./Data/ubIonosphere.csv', index_col = 0)

In [4]:
#dimension
print('dimension', ubIonosphere.shape)
#Table
print(ubIonosphere['Class'].value_counts())

dimension (351, 33)
0    225
1    126
Name: Class, dtype: int64


In [5]:
y = ubIonosphere['Class']
x = ubIonosphere.copy().drop(['Class'], axis = 1)

In [6]:
# 方法1: "replicates randomly" some instances from the "minority" class
# Hint: over-sampling

max_size = ubIonosphere['Class'].value_counts().max()

for class_index, group in ubIonosphere.groupby('Class'):
    overData = ubIonosphere.append(group.sample(max_size - len(group), replace = True))
    
print(overData['Class'].value_counts())

1    225
0    225
Name: Class, dtype: int64


In [7]:
# 方法2: "removes randomly" some instances from the "majority (negative)" class
# Hint: under-sampling
underData = pd.DataFrame()
min_size = ubIonosphere['Class'].value_counts().min()

for class_index, group in ubIonosphere.groupby('Class'):
    underData = underData.append(group.sample(min_size))
    
print(underData['Class'].value_counts())

1    126
0    126
Name: Class, dtype: int64


In [8]:
# 方法3: "synthetic minority over-sampling technique"
# Hint: "SMOTE", generates new examples by filling empty areas among the positive instances

smote_x, smote_y = SMOTE().fit_sample(x, y)
bdata = pd.DataFrame(data = smote_x, columns = x.columns)
bdata['Class'] = smote_y
print(bdata['Class'].value_counts())

1    225
0    225
Name: Class, dtype: int64


In [9]:
# Compare the Performances using SVM:
# 將 ubIonosphere dataset 切分成 training set + test set:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5)

In [10]:
# 方法1: 單純使用 SVM
# Hint: x.train, y.train
model1 = SVC().fit(x_train, y_train)
y_pred1 = model1.predict(x_test)
confusion_matrix(y_test, y_pred1)

/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([[107,   1],
       [ 13,  55]])

In [11]:
# 方法2: SMOTE + SVM
# Hint: balancedData$X, balancedData$Y
smote_x, smote_y = SMOTE().fit_sample(x_train, y_train)
model2 = SVC().fit(smote_x, smote_y)
y_pred2 = model2.predict(x_test)
confusion_matrix(y_test, y_pred2)

/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([[104,   4],
       [ 13,  55]])